In [3]:

!git clone https://github.com/karpathy/minGPT.git  -q
%cd minGPT
!pip install -e . -qqq
!pip install transformers[torch] -qqq

c:\Users\au616584\OneDrive - Aarhus Universitet\Datalogi\PhD\Faster-GPT-generation\minGPT


fatal: destination path 'minGPT' already exists and is not an empty directory.


In [4]:
import torch
from mingpt.model import GPT
print("step")
from transformers import GPT2Tokenizer, GPT2LMHeadModel
print("step 2")
from mingpt.utils import set_seed
print("step 3")
from mingpt.bpe import BPETokenizer
set_seed(3407)
print("step 4")
model_type = 'gpt2' 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT.from_pretrained(model_type)
model.to(device)
model.eval()

step
step 2
step 3
step 4
number of parameters: 124.44M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): 

In [9]:
from mingpt.bpe import BPETokenizer
tokenizer = BPETokenizer()

num_samples = 1

x   = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
x   = x.expand(num_samples, -1).to(device)
y   = model.generate(x, max_new_tokens=3, do_sample=True, top_k=40)
out = tokenizer.decode(y[0].cpu().squeeze())

print(out)

Hi at the end


In [40]:
from torch.nn import functional as F
import torch as T

def sample_tokens(logits: T.Tensor, z: T.Tensor) -> T.Tensor:
    cdf = T.cumsum(T.nn.functional.softmax(logits, dim=1), dim=1)
    return (cdf < z[:, None]).sum(1) # holy fucking douglas

# def unsample_z(logits: T.Tensor, tokens: T.Tensor) -> T.Tensor:
#     cdf = T.cumsum(T.cat([T.zeros_like(logits[:, 0, None]), T.nn.functional.softmax(logits, dim=1)], dim=1), dim=1)
#     idx = T.arange(tokens.shape[0])
#     return T.stack([cdf[idx, tokens], cdf[idx, tokens+1]])


def unsample_z(logits: T.Tensor, tokens: T.Tensor) -> T.Tensor:
    cdf = T.cumsum(T.cat([T.zeros_like(logits[:, 0, None]), T.nn.functional.softmax(logits, dim=1)], dim=1), dim=1)
    idx = T.arange(tokens.shape[0])
    return T.stack([cdf[idx, tokens], cdf[idx, tokens+1]])



# @torch.no_grad()
# def generate(self, idx, Z, max_new_tokens):

#         for i in range(max_new_tokens):
#             logits, _ = self(idx)
#             logits    = logits[:, -1, :] 
#             idx_next  = sample_tokens(logits, Z[i].view(1,) ).view(1,1)

#             redo = unsample_z(logits, idx_next)
#             assert redo[0] <= Z[i] <=  redo[1]

#             idx       = torch.cat((idx, idx_next), dim=1)
#             #print(redo[0], Z[i], redo[1])

#         return idx
# @torch.no_grad()
# def generate(self, idx, Z, max_new_tokens):
#   diffs = []
#   for i in range(max_new_tokens):
#       logits, _ = self(idx)
#       logits    = logits[:, -1, :] 
#       idx_next  = sample_tokens(logits, Z[i].view(1,) ).view(1,1)

#       cdf = T.cumsum(T.nn.functional.softmax(logits, dim=1), dim=1).view(-1)

#       diff = (cdf[1:]-cdf[:-1]).reshape(-1).detach().cpu().numpy()
#       diffs.append((diff, idx_next))

#       redo = unsample_z(logits, idx_next)
#       assert redo[0] < Z[i] <  redo[1]

#       idx       = torch.cat((idx, idx_next), dim=1)

#   return idx, diffs

@torch.no_grad()
def generate(self, idx, Z, max_new_tokens):
  diffs = []
  batch_size = idx.shape[0]
  for i in range(max_new_tokens):
      logits, _ = self(idx)
      logits    = logits[:, -1, :] 
      idx_next  = sample_tokens(logits, Z[i].view(1,) ).view(batch_size,1)

      cdf = T.cumsum(T.nn.functional.softmax(logits, dim=1), dim=1).view(-1)

      diff = (cdf[1:]-cdf[:-1]).reshape(-1).detach().cpu().numpy()
      diffs.append((diff, idx_next))

      redo = unsample_z(logits, idx_next)

      idx       = torch.cat((idx, idx_next), dim=1)

  return idx, diffs
# generate
max_new_tokens = 2
n = 50257

torch.manual_seed(42)
Z = torch.rand((max_new_tokens), device=device)

num_samples = 1
x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
x = x.expand(num_samples, -1).to(device)

y, diff1 = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
out = tokenizer.decode(y[0].cpu().squeeze())
print(out)

# y = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
# out = tokenizer.decode(y[0].cpu().squeeze())
# print(out)

logits, _ = model(y)
print(logits.shape, y.shape)
#idx_next  = sample_tokens(logits, Z.view(1,) ).view(1,1)

logits, _ = model(y)
redo = unsample_z(logits[0, 0:-1], y[0, 1:]) # this is the inverse!  F(logits, y) 
print(redo)
print(Z)

Hi four centuries
torch.Size([1, 3, 50257]) torch.Size([1, 3])
tensor([[0.6531, 0.9343],
        [0.6531, 0.9357]], grad_fn=<StackBackward0>)
tensor([0.8823, 0.9150])


In [52]:
# torch.manual_seed(11)
# Z = torch.rand((max_new_tokens), device=device)
# print(f"Z_j = {Z[0]}")
# print(f"Z = {Z}")
# x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
# print(x)
# y_z = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
# print(y_z)
# print(tokenizer.decode(y_z[0].cpu().squeeze()))
# x_ = x.clone()
# for p in range(max_new_tokens):
#     for i in range(n):
#         print(i, end='\r')
#         x_prime = torch.cat((x_, torch.tensor([[i]], dtype=torch.long)), dim=1)
#         # x_prime = torch.cat((x_prime, torch.tensor([[i]], dtype=torch.long)), dim=1)
#         logits, _ = model(x_prime)
#         redo = unsample_z(logits[0, 0:-1], x_prime[0, 1:]) # this is the inverse!  F(logits, y)
#         # print(redo[0], Z[p], redo[1][p],end='\r')
#         if redo[0][p] <= Z[p] <= redo[1][p]:
#             print(i, redo[0], Z, redo[1])
#             x_ = x_prime
#             break
#This is doing the same as sample tokens!!!

In [53]:
#do the same, but iterate over everything in redo to find the right one
torch.manual_seed(12)
max_new_tokens = 2
Z = torch.rand((max_new_tokens), device=device)
print(f"Z_j = {Z[0]}")
print(f"Z = {Z}")
x = torch.tensor([tokenizer.encoder.encode('20 plus 1 equals')], dtype=torch.long)
print(x)
y_z = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
print(y_z)
print(tokenizer.decode(y_z[0].cpu().squeeze()))
torch.manual_seed(11)

x_ = x.clone()
for p in range(max_new_tokens):
        # x_prime = torch.cat((x_prime, torch.tensor([[i]], dtype=torch.long)), dim=1)
    logits, _ = model(x_)
    next_word = sample_tokens(logits[:, -1, :], Z[p].view(1,) ).view(1,1)
    print(next_word)
    x_ = torch.cat((x_, next_word), dim=1)    
        # redo = unsample_z(logits[0, 0:-1], x_prime[0, 1:]) # this is the inverse!  F(logits, y)
        # # print(redo[0], Z[p], redo[1][p],end='\r')
        # if redo[0][p] <= Z[p] <= redo[1][p]:
        #     print(i, redo[0], Z, redo[1])
        #     x_ = x_prime
        #     break
print(tokenizer.decode(x_[0].cpu().squeeze()))

Z_j = 0.46568745374679565
Z = tensor([0.46568745374679565430, 0.23276680707931518555])
tensor([[ 1238,  5556,   352, 21767]])
(tensor([[ 1238,  5556,   352, 21767,   513,    13]]), [(array([6.5465152e-05, 1.4426332e-06, 7.4970594e-06, ..., 0.0000000e+00,
       0.0000000e+00, 2.8073788e-05], dtype=float32), tensor([[513]])), (array([2.9321283e-03, 8.7277498e-05, 1.1612498e-03, ..., 1.1920929e-07,
       0.0000000e+00, 1.1499524e-03], dtype=float32), tensor([[13]]))])
20 plus 1 equals 3.
tensor([[513]])
tensor([[13]])
20 plus 1 equals 3.


In [54]:
torch.manual_seed(11)
Z = torch.rand((1), device=device)
print(f"Z_j = {Z[0]}")
print(f"Z = {Z}")
x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
print(x)
x_ = x.clone()
for i in range(550,551):
    print(i, end='\r')
    x_prime = torch.cat((x_, torch.tensor([[tokenizer.encoder.encoder['.']]], dtype=torch.long)), dim=1)
    # x_prime = torch.cat((x_prime, torch.tensor([[i]], dtype=torch.long)), dim=1)
    logits, _ = model(x_prime)
    redo = unsample_z(logits[0, 0:-1], x_prime[0, 1:]) # this is the inverse!  F(logits, y)
    # print(redo[0], Z[p], redo[1][p],end='\r')
    if redo[0] <= Z <= redo[1]:
        print(i, redo[0], Z, redo[1])
        x_ = x_prime
        break


Z_j = 0.14904171228408813
Z = tensor([0.14904171228408813477])
tensor([[17250]])
550 tensor([0.11745364964008331299], grad_fn=<SelectBackward0>) tensor([0.14904171228408813477]) tensor([0.19352594017982482910], grad_fn=<SelectBackward0>)


In [55]:
torch.manual_seed(45)


max_new_tokens = 5
n = 50257
Z = torch.rand((max_new_tokens), device=device)

num_samples = 1
x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
x = x.expand(num_samples, -1).to(device)
y,_ = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
print(f"y: {y}")
out = tokenizer.decode(y[0].cpu().squeeze())
print(out)

logits, _ = model(y) 
print(logits.shape)
inverse_ranges = unsample_z(logits[0,0:-1], y[0,1:])
assert(torch.all(torch.logical_and(inverse_ranges[0]<=Z, Z<=inverse_ranges[1])))
print(f"Z:{Z}, inverse range: {inverse_ranges}")
#Test that a different number in the interval is mapped to the same value
Z_prime = inverse_ranges[0]+0.000001
# print(Z_prime)
y_prime,_ = generate(model, x, Z_prime, max_new_tokens = max_new_tokens)
out_prime = tokenizer.decode(y_prime[0].cpu().squeeze())
print(out_prime)

y: tensor([[17250,    13, 31342,   314,  3285,   262]])
Hi. Personally I hear the
torch.Size([1, 6, 50257])
Z:tensor([0.18694752454757690430, 0.96132838726043701172, 0.68344897031784057617,
        0.89879584312438964844, 0.05050849914550781250]), inverse range: tensor([[0.11745364964008331299, 0.96123254299163818359, 0.57818794250488281250,
         0.89862650632858276367, 0.04298967868089675903],
        [0.19352594017982482910, 0.96133017539978027344, 0.88418436050415039062,
         0.89886742830276489258, 0.08601097762584686279]],
       grad_fn=<StackBackward0>)
Hi. Personally I hear the


In [56]:
#Test how changing a single word affects the ranges
y_lwc = y.clone() #last word changed
y_lwc[0][-1] = y_lwc[0][-1]+10
print(y_lwc)
out_lwc = tokenizer.decode(y_lwc[0].cpu().squeeze())
print(out_lwc)
logits_lwc,_ = model(y_lwc)
inverse_ranges_lwc = unsample_z(logits_lwc[0,0:-1], y_lwc[0, 1:])
print(inverse_ranges_lwc)
#notice that in this case all the previous words are not changed. So you could do it element by element from the back?
y_fwc = y.clone() #last word changed
y_fwc[0][1] = y_fwc[0][1]+20
print(y_fwc)
out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
print(out_fwc)
logits_fwc,_ = model(y_fwc)
inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
print(inverse_ranges_fwc)
y_fwc[0][1] = y_fwc[0][1]+21
print(y_fwc)
out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
print(out_fwc)
logits_fwc,_ = model(y_fwc)
inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
print(inverse_ranges_fwc)
y_fwc[0][1] = y_fwc[0][1]+22
print(y_fwc)
out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
print(out_fwc)
logits_fwc,_ = model(y_fwc)
inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
print(inverse_ranges_fwc)
#Changing the first word changes all the probabilities of the subsequent words.

tensor([[17250,    13, 31342,   314,  3285,   272]])
Hi. Personally I hearan
tensor([[0.11745364964008331299, 0.96123254299163818359, 0.57818794250488281250,
         0.89862650632858276367, 0.08682110905647277832],
        [0.19352594017982482910, 0.96133017539978027344, 0.88418436050415039062,
         0.89886742830276489258, 0.08682221919298171997]],
       grad_fn=<StackBackward0>)
tensor([[17250,    33, 31342,   314,  3285,   262]])
HiB Personally I hear the
tensor([[0.23588979244232177734, 0.94166392087936401367, 0.53124368190765380859,
         0.89211815595626831055, 0.04916113242506980896],
        [0.23697061836719512939, 0.94166469573974609375, 0.77233487367630004883,
         0.89229446649551391602, 0.10557980090379714966]],
       grad_fn=<StackBackward0>)
tensor([[17250,    54, 31342,   314,  3285,   262]])
HiW Personally I hear the
tensor([[0.25298658013343811035, 0.95035153627395629883, 0.57684189081192016602,
         0.89739674329757690430, 0.05235633626580238342],
  

In [57]:
for i in range(20):
    y_fwc = y.clone() #last word changed
    y_fwc[0][1] = y_fwc[0][1]+i
    print(y_fwc)
    out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
    print(out_fwc)
    logits_fwc,_ = model(y_fwc)
    inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
    print(inverse_ranges_fwc[0][1], inverse_ranges_fwc[1][1])

tensor([[17250,    13, 31342,   314,  3285,   262]])
Hi. Personally I hear the
tensor(0.96123254299163818359, grad_fn=<SelectBackward0>) tensor(0.96133017539978027344, grad_fn=<SelectBackward0>)
tensor([[17250,    14, 31342,   314,  3285,   262]])
Hi/ Personally I hear the
tensor(0.83557391166687011719, grad_fn=<SelectBackward0>) tensor(0.83557397127151489258, grad_fn=<SelectBackward0>)
tensor([[17250,    15, 31342,   314,  3285,   262]])
Hi0 Personally I hear the
tensor(0.94795876741409301758, grad_fn=<SelectBackward0>) tensor(0.94796025753021240234, grad_fn=<SelectBackward0>)
tensor([[17250,    16, 31342,   314,  3285,   262]])
Hi1 Personally I hear the
tensor(0.94894570112228393555, grad_fn=<SelectBackward0>) tensor(0.94894880056381225586, grad_fn=<SelectBackward0>)
tensor([[17250,    17, 31342,   314,  3285,   262]])
Hi2 Personally I hear the
tensor(0.93389606475830078125, grad_fn=<SelectBackward0>) tensor(0.93389958143234252930, grad_fn=<SelectBackward0>)
tensor([[17250,    18, 31

In [58]:
def nn(x):   
  emb_dim  = model.transformer.wte.weight.shape[1]
  return torch.argmin( torch.sum( (model.transformer.wte.weight.reshape(1, -1,emb_dim) - x.reshape(-1, 1, emb_dim))**2, axis=2), axis=1)
print(model.transformer.wte.weight.shape[1])
def extracted_forward(embedding, t, device):
  pos    = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)
  # the rest is just normal forward pass 
  pos_emb = model.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
  #x = model.transformer.drop(tok_emb + pos_emb)
  x = embedding + pos_emb
  for block in model.transformer.h:
      x = block(x)
  x = model.transformer.ln_f(x)
  return model.lm_head(x) # output of forward pass 
  
torch.manual_seed(42)
max_new_tokens, n = 5, 50257
Z = torch.rand((max_new_tokens), device=device)
prompt  = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long, device=device)
generated_target,_ = generate(model, prompt, Z, max_new_tokens = max_new_tokens)
target_idx = generated_target
b, t    = target_idx.size()

print("target_idx: ", target_idx)
logits, _ = model(target_idx)
print("unsampled: ", unsample_z(logits[0,0:-1], target_idx[0,1:]))
print("unsampled: ", unsample_z(logits[0,1:], target_idx[0,1:]))
print("Z:", Z)
print(generated_target)


target_emb = model.transformer.wte(target_idx)
target_emb_grad = target_emb.clone().detach().requires_grad_(True) # then pick idx to be nearest neighboar to token emb! 
print(f"target sentence: ", tokenizer.decode(target_idx.cpu().squeeze()))


scrambled_idx = target_idx.clone()
scrambled_idx[0][-2] = scrambled_idx[0][-2]+2
print(f"scrambled idx, {scrambled_idx}")
print(f"scrambled sentence: ", tokenizer.decode(scrambled_idx.cpu().squeeze()))
scrambled_emb = model.transformer.wte(scrambled_idx)
scrambled_emb_grad = scrambled_emb.clone().detach().requires_grad_(True)

adam = torch.optim.Adam([scrambled_emb_grad], lr=0.01) 
for i in range(1): 
  idx_guess = nn(scrambled_emb_grad).view(-1)
  adam.zero_grad()
   # just do forward pass 
  W_logits = extracted_forward(scrambled_emb_grad, t, device) # output of forward pass 
  enc_W_logits = unsample_z(W_logits[0,0:-1], idx_guess[1:] )
  # prompt = torch.tensor([[tokenizer.encoder.encoder['What']]], dtype=torch.long, device=device)
  # y_prime = generate(model, prompt, (enc_W_logits[0]+0.0001), max_new_tokens = max_new_tokens)
  # out_prime = tokenizer.decode(y_prime[0].cpu().squeeze())
  loss = torch.mean((Z - enc_W_logits[0]).abs()) +  torch.mean((Z - enc_W_logits[1]).abs())

  # loss = (Z[-1] - enc_W_logits[0][-1])**2+ (Z[-1] - enc_W_logits[1][-1])**2
  print(f"min: {enc_W_logits[0]}, max: {enc_W_logits[1]}")

  print(scrambled_emb_grad[0][4])
  print("\r[%i / %i] "%(i+1, 10000),
        torch.mean((target_emb_grad[0][-2]- scrambled_emb_grad[0][-2]).abs()).data.cpu().numpy(), 
        loss.data.cpu().numpy(), 
        torch.set_printoptions(precision=20),
        print("full max_min: ", enc_W_logits[0][-2], enc_W_logits[1][-2]),
        # idx_guess.cpu().numpy(), 
        tokenizer.decode(idx_guess.cpu().squeeze()), end="")

  if i == 0: print("")
  loss.backward()
  # assert(torch.all(scrambled_emb_grad.grad[0][-1]!=0))
  scrambled_emb_grad.grad[:,:-2,:]=0
  # assert(torch.any(scrambled_emb_grad.grad[0][-1]!=0))  
  adam.step()
  adam.zero_grad()

768
target_idx:  tensor([[17250, 10109,  5434,   198,   198,  1890]])
unsampled:  tensor([[8.82268548011779785156e-01, 9.14999902248382568359e-01,
         3.70913326740264892578e-01, 4.86702338093891739845e-04,
         3.86159121990203857422e-01],
        [8.82275164127349853516e-01, 9.15004551410675048828e-01,
         4.37913805246353149414e-01, 9.99266922473907470703e-01,
         3.91458749771118164062e-01]], grad_fn=<StackBackward0>)
unsampled:  tensor([[9.44543540477752685547e-01, 8.75234127044677734375e-01,
         4.86702338093891739845e-04, 1.89616650342941284180e-01,
         5.94471871852874755859e-01],
        [9.44699108600616455078e-01, 8.75584304332733154297e-01,
         9.99266922473907470703e-01, 1.89672201871871948242e-01,
         5.94472706317901611328e-01]], grad_fn=<StackBackward0>)
Z: tensor([0.88226926326751708984, 0.91500395536422729492, 0.38286375999450683594,
        0.95930564403533935547, 0.39044821262359619141])
tensor([[17250, 10109,  5434,   198,   1

In [59]:
#Using the fact that we have a limited search space, can we just iterate over all the possible words and find the one that minimizes the loss?
# torch.manual_seed(42)
max_new_tokens, n = 5, 50257
Z = torch.rand((max_new_tokens), device=device)
prompt  = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long, device=device)
generated_target,_ = generate(model, prompt, Z, max_new_tokens = max_new_tokens)
target_idx = generated_target
b, t    = target_idx.size()

print("target_idx: ", target_idx)
logits, _ = model(target_idx)
print("unsampled: ", unsample_z(logits[0,0:-1], target_idx[0,1:]))
print("Z:", Z)
print(generated_target)

range_shuffled = torch.randperm(n)

scrambled_idx = target_idx.clone()
scrambled_idx[0][-2] = scrambled_idx[0][-2]+2
for i in range(1,max_new_tokens+1):
    logits_fwc,_ = model(scrambled_idx)
    inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], scrambled_idx[0, 1:])
    print(inverse_ranges_fwc[0][i-1], inverse_ranges_fwc[1][i-1])
    # if Z[i-1] > inverse_ranges_fwc[0][i-1] and Z[i-1] < inverse_ranges_fwc[1][i-1]:
    #     print(Z[i])
    #     print("Z[i] < inverse_ranges_fwc[0][i]")
    #     continue
    print(inverse_ranges_fwc[0][i-1], inverse_ranges_fwc[1][i-1])
    for j in range_shuffled:
        print("in the loop with id: ", i)
        scrambled_idx[0][i] = j
        logits_fwc,_ = model(scrambled_idx)
        inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], scrambled_idx[0, 1:])
        if Z[i-1] < inverse_ranges_fwc[1][i-1] or Z[i-1] > inverse_ranges_fwc[0][i-1]:
            print("Z[i] < inverse_ranges_fwc[0][i]")
            scrambled_idx[0][i-1] = j
            break
print(tokenizer.decode(scrambled_idx.cpu().squeeze()))
print(scrambled_idx)



target_idx:  tensor([[17250,   645,    11,  2035,  9959,    27]])
unsampled:  tensor([[0.60023993253707885742, 0.04278983548283576965, 0.79356706142425537109,
         0.94076949357986450195, 0.06635893881320953369],
        [0.60114079713821411133, 0.34722551703453063965, 0.79367482662200927734,
         0.94077199697494506836, 0.14652654528617858887]],
       grad_fn=<StackBackward0>)
Z: tensor([0.60089534521102905273, 0.25657248497009277344, 0.79364132881164550781,
        0.94077146053314208984, 0.13318592309951782227])
tensor([[17250,   645,    11,  2035,  9959,    27]])
tensor(0.60023993253707885742, grad_fn=<SelectBackward0>) tensor(0.60114079713821411133, grad_fn=<SelectBackward0>)
tensor(0.60023993253707885742, grad_fn=<SelectBackward0>) tensor(0.60114079713821411133, grad_fn=<SelectBackward0>)
in the loop with id:  1
Z[i] < inverse_ranges_fwc[0][i]
tensor(0.01872552931308746338, grad_fn=<SelectBackward0>) tensor(0.11563965678215026855, grad_fn=<SelectBackward0>)
tensor(0.0187

In [60]:
from datasets import load_dataset
dataset = load_dataset("anthropic/hh-rlhf")

Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00,  5.43it/s]


In [61]:
import random
import torch
from datasets import load_dataset
import pickle
vocab_size = 50257

def corrupt_sentence(sentence, k, tokenizer, device):
    sentence_encoded = torch.tensor([[tokenizer.encoder.encode(sentence)]], dtype=torch.long, device=device)
    corruption_indices = random.sample(range(0, len(sentence_encoded[0][0])), k)
    corrupted_encoding = sentence_encoded.clone().detach()

    for j in corruption_indices:
        corrupted_encoding[0][0][j] = (random.sample(range(0, vocab_size), 1)[0])

    corrupted_sentence = tokenizer.decode(corrupted_encoding[0].cpu().squeeze())
    return corrupted_sentence

def save_sentences(sentences, filename):
    with open(filename, 'wb') as f:
        pickle.dump(sentences, f)

def load_sentences(filename):
    with open(filename, 'rb') as f:
        sentences = pickle.load(f)
    return sentences

dataset = load_dataset("anthropic/hh-rlhf")

# Set the corruption parameter
k = 10

# Apply corruption to the dataset
original_and_corrupted_sentences = []

for i, example in enumerate(dataset['train']):
    if i >= 10:
        break
    original_sentence = example['chosen']
    corrupted_sentence = corrupt_sentence(original_sentence, k, tokenizer, device)
    original_and_corrupted_sentences.append((original_sentence, corrupted_sentence))

# Save the sentences
save_sentences(original_and_corrupted_sentences, 'corrupted_sentences.pkl')

# Load the sentences
loaded_sentences = load_sentences('corrupted_sentences.pkl')

# Print an example
print(dataset['train'][1]['chosen'])
print(loaded_sentences[1][0][:1000])
print("**************")
print(loaded_sentences[1][1])

Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00, 29.83it/s]




Human: What kind of noises did dinosaurs make?

Assistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be

Human: yes they did

Assistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.

Human: you cant read

Assistant: You can read?


Human: What kind of noises did dinosaurs make?

Assistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be

Human: yes they did

Assistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.

Human: you cant read

Assistant: You can read?
**************


Human: What kind of noises did dinosaurs make?

 became doesn Humans and dinosaurs didn’t live htt the same time plugs so it’s really hard to s

In [62]:
import random
import torch
import re
from datasets import load_dataset
from transformers import AutoTokenizer
import pickle

def corrupt_sentence(sentence, k, tokenizer, device):
    sentence_encoded = torch.tensor([[tokenizer.encoder.encode(sentence)]], dtype=torch.long, device=device)
    corruption_indices = random.sample(range(0, len(sentence_encoded[0][0])), k)
    corrupted_encoding = sentence_encoded.clone().detach()

    for j in corruption_indices:
        corrupted_encoding[0][0][j] = (random.sample(range(0, vocab_size), 1)[0])

    corrupted_sentence = tokenizer.decode(corrupted_encoding[0].cpu().squeeze())
    return corrupted_sentence

def save_sentences(sentences, filename):
    with open(filename, 'wb') as f:
        pickle.dump(sentences, f)

def load_sentences(filename):
    with open(filename, 'rb') as f:
        sentences = pickle.load(f)
    return sentences

dataset = load_dataset("anthropic/hh-rlhf")

# Set the corruption parameter
k = 10

# Apply corruption to the dataset
original_and_corrupted_sentences = []

for i, example in enumerate(dataset['train']):
    if i >= 10:
        break
    original_sentence = example['chosen']
    assistant_pattern = re.compile(r'Assistant:(.*?)Human:', re.DOTALL)
    assistant_response = re.search(assistant_pattern, original_sentence)
    if assistant_response:
        original_assistant_response = assistant_response.group(1)
        corrupted_assistant_response = corrupt_sentence(original_assistant_response, k, tokenizer, device)
        corrupted_sentence = original_sentence.replace(original_assistant_response, corrupted_assistant_response)
        original_and_corrupted_sentences.append((original_sentence, corrupted_sentence))

# Save the sentences
save_sentences(original_and_corrupted_sentences, 'corrupted_sentences.pkl')

# Load the sentences
loaded_sentences = load_sentences('corrupted_sentences.pkl')

# Print an example
print(dataset['train'][0]['chosen'])
print("**************")
print(loaded_sentences[0][1])


Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00, 26.62it/s]



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.
**************


Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger covered crap, fuck, shit, bitch, turd, shithead, shitbag, sc propellum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, but Contributole, poop Kyoto jackass, Eternalocksucker, asshole, goddamn, piss Das sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jiz

In [125]:
import re
from datasets import load_dataset
import pickle

def save_prompts(prompts, filename):
    with open(filename, 'wb') as f:
        pickle.dump(prompts, f)

def load_prompts(filename):
    with open(filename, 'rb') as f:
        prompts = pickle.load(f)
    return prompts

dataset = load_dataset("anthropic/hh-rlhf")

# Extract the first human prompt from each sample
human_prompts = []

for i, example in enumerate(dataset['train']):
    if i >= 3:
        break
    original_sentence = example['chosen']
    human_pattern = re.compile(r'Human:(.*?)(Assistant:|$)', re.DOTALL)
    human_prompt = re.search(human_pattern, original_sentence)
    if human_prompt:
        human_prompts.append(human_prompt.group(1).strip())

# Save the human prompts
save_prompts(human_prompts, 'human_prompts.pkl')

# Load the human prompts
loaded_prompts = load_prompts('human_prompts.pkl')

# Print an example
print(loaded_prompts[0])

Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00,  6.06it/s]

What are some cuss words in english?


In [126]:
prompts = load_prompts('human_prompts.pkl')
num_tokens = 10
Z = torch.rand((num_tokens), device=device)

for prompt in prompts:
    print(prompt)
    
    encoded  = torch.tensor([tokenizer.encoder.encode(prompt)], dtype=torch.long, device=device)
    generated,_ = generate(model, encoded, Z, max_new_tokens = num_tokens)

    print(tokenizer.decode(generated[0]))
    

What are some cuss words in english?
What are some cuss words in english?

Currently, collups ideas are from within
What kind of noises did dinosaurs make?
What kind of noises did dinosaurs make? (Or rather, street noises?) And which sort
If you were going to steal from a convenience store, do you think it would be better in the morning or at night?
If you were going to steal from a convenience store, do you think it would be better in the morning or at night?

Put in short script how do you feel


In [87]:
import random
def corrupt_generated(generated, k, tokenizer, device):
    num_tokens = len(generated[0])
    k = min(k, num_tokens)  
    corruption_indices = random.sample(range(0, num_tokens), k)
    corrupted_generated = generated.clone().detach()
    print(corruption_indices)
    for j in corruption_indices:
        corrupted_generated[0][j] = (random.sample(range(0, vocab_size), 1)[0])

    return corrupted_generated, corruption_indices

In [128]:
prompts = load_prompts('human_prompts.pkl')
num_tokens = 10
num_corruptions = 2

Z = torch.rand((num_tokens), device=device)
vocab_size = 50257
single_prompt = None
single_generated_corrrupted = None
single_generated = None
for prompt in prompts:
    print(prompt)
    
    encoded  = torch.tensor([tokenizer.encoder.encode(prompt)], dtype=torch.long, device=device)
    single_prompt = encoded
    prompt_len = len(encoded[0])
    generated,_ = generate(model, encoded, Z, max_new_tokens = num_tokens)
    single_generated = generated
    print(generated)
    generated_corrupted = corrupt_generated(generated[0][prompt_len:], num_corruptions, tokenizer, device)
    full_corrupted = torch.cat((encoded[0],generated_corrupted)).unsqueeze(dim=0)
    print(generated_corrupted.shape)
    single_generated_corrrupted = full_corrupted
    print(full_corrupted.shape)
    print(tokenizer.decode(generated[0]))
    print("*****")
    print(tokenizer.decode(full_corrupted[0]))

print("We're working with a single corrupted prompt:", single_generated_corrrupted)

What are some cuss words in english?
tensor([[ 2061,   389,   617,   269,  1046,  2456,   287, 46932,    30, 10358,
            77,   470,  5876,  1037,  2130,   503,  5633,  3740,  1378]])
[0, 1]


TypeError: expected Tensor as element 1 in argument 0, but got tuple

In [12]:
def unsample_z(logits: T.Tensor, tokens: T.Tensor) -> T.Tensor:
    cdf = T.cumsum(T.cat([T.zeros_like(logits[:, 0, None]), T.nn.functional.softmax(logits, dim=1)], dim=1), dim=1)
    idx = T.arange(tokens.shape[0])
    return T.stack([cdf[idx, tokens], cdf[idx, tokens+1]])

In [55]:
def loss(target_Z, encoded_range):
    b = 1 - 1.*((encoded_range[0] <= target_Z) & (target_Z < encoded_range[1]))
    a = torch.max( torch.abs(encoded_range[0] - target_Z), torch.abs(encoded_range[1] - target_Z)) * b
    return a.detach().cpu().numpy()


logits, _ = model(single_generated_corrrupted)
interval = unsample_z(logits[0,0:-1], single_generated_corrrupted[0,1:])

_loss = loss(Z, interval[:,-num_tokens:])
print("Loss: ", end="")
for c, l in zip(tokenizer.decode(full_corrupted[0]).split(" ")[-Z.shape[0]:], _loss):
    print("{:.2f} ".format(l), end="")
print()


NameError: name 'single_generated_corrrupted' is not defined

In [103]:
logits, _ = model(single_generated)
interval = unsample_z(logits[0,0:-1], single_generated[0,1:])

_loss = loss(Z, interval[:, -num_tokens:])
print("Loss: ", end="")
for c, l in zip(tokenizer.decode(full_corrupted[0]).split(" ")[-Z.shape[0]:], _loss):
    print("{:.2f} ".format(l), end="")
print()

Loss: 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 


In [25]:
import numpy as np
prompts = load_prompts('human_prompts.pkl')
num_tokens = 10
num_corruptions = 2

Z = torch.rand((num_tokens), device=device)
vocab_size = 50257
acc_loss_corrupted = []
acc_loss_uncorrupted = []
model_sizes = ['gpt2', 'gpt2-medium', 'gpt2-large']#, 'gpt2-xl']
for i, model_size in enumerate(model_sizes):
    model = GPT.from_pretrained(model_type)
    acc_loss_corrupted.append(0)
    acc_loss_uncorrupted.append(0)
    for prompt in prompts:
        print(prompt)
        
        encoded  = torch.tensor([tokenizer.encoder.encode(prompt)], dtype=torch.long, device=device)
        prompt_len = len(encoded[0])
        generated,_ = generate(model, encoded, Z, max_new_tokens = num_tokens)
        generated_corrupted, indices = corrupt_generated(generated[0][prompt_len:], num_corruptions, tokenizer, device)
        full_corrupted = torch.cat((encoded[0],generated_corrupted)).unsqueeze(dim=0)
        logits,_ = model(full_corrupted)
        interval = unsample_z(logits[0,0:-1], full_corrupted[0,1:])
        _loss = loss(Z, interval[:,-num_tokens:])
        acc_loss_corrupted[i] += _loss[indices].sum()
        mask = np.ones_like(_loss, dtype=bool)
        mask[indices] = False
        acc_loss_uncorrupted[i] = _loss[mask].sum()
print("Loss on corrupted: ", acc_loss_corrupted)
print("Loss on not corrupted: ", acc_loss_uncorrupted)
print("We're working with a single corrupted prompt:", single_generated_corrrupted)

number of parameters: 124.44M
What are some cuss words in english?


ValueError: Sample larger than population or is negative

In [6]:
import re
from datasets import load_dataset
import pickle

def save_prompts(prompts, filename):
    with open(filename, 'wb') as f:
        pickle.dump(prompts, f)

def load_prompts(filename):
    with open(filename, 'rb') as f:
        prompts = pickle.load(f)
    return prompts

dataset = load_dataset("anthropic/hh-rlhf")

# Extract the whole prompt from each sample
whole_prompts = []

for i, example in enumerate(dataset['train']):
    if i >= 3:
        break
    whole_prompt = example['chosen']
    whole_prompts.append(whole_prompt)

# Save the whole prompts
save_prompts(whole_prompts, 'whole_prompts.pkl')

# Load the whole prompts
loaded_prompts = load_prompts('whole_prompts.pkl')

# Print an example
print(loaded_prompts[0])


Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00,  5.73it/s]



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.


In [21]:
def print_encoded(encoding):
    print(tokenizer.decode(encoding[0].cpu().squeeze()))

In [90]:
import numpy as np
prompts = load_prompts('human_prompts.pkl')
num_tokens = 10
num_corruptions = 2

Z = torch.rand((num_tokens), device=device)
vocab_size = 50257
acc_loss_corrupted = []
acc_loss_uncorrupted = []
acc_loss_original_corrupted = []
acc_loss_original_uncorrupted = []
model_sizes = ['gpt2', 'gpt2-medium', 'gpt2-large']
for i, model_size in enumerate(model_sizes):
    model = GPT.from_pretrained(model_type)
    model.eval()
    acc_loss_corrupted.append(0)
    acc_loss_uncorrupted.append(0)
    acc_loss_original_corrupted.append(0)
    acc_loss_original_uncorrupted.append(0)
    
    for prompt in prompts:
        print(prompt)        
        encoded  = torch.tensor([tokenizer.encoder.encode(prompt)], dtype=torch.long, device=device)
        prompt_len = len(encoded[0])
        logits,_ = model(encoded)
        z = unsample_z(logits[0, 0:-1], encoded[0, 1:])
        z_ = (z[0]+z[1])/2
        generated,_ = generate(model, encoded[:,:1], z_, max_new_tokens = prompt_len-1)
        assert torch.equal(encoded[0,1:], generated[0][1:])
        
        corrupted_prompt, indices = corrupt_generated(encoded[:,1:], num_corruptions, tokenizer, device)
        full_corrupted = torch.cat((encoded[:,:1],corrupted_prompt),dim=1)
        logits,_ = model(full_corrupted)
        interval = unsample_z(logits[0,0:-1], full_corrupted[0,1:])
        _loss = loss(z_, interval)
        print(f"loss: {_loss}")
        acc_loss_corrupted[i] += _loss[indices].sum()
        mask = np.ones_like(_loss, dtype=bool)
        mask[indices] = False
        acc_loss_uncorrupted[i] = _loss[mask].sum()
        
        logits,_ = model(encoded)
        interval = unsample_z(logits[0,0:-1], encoded[0,1:])
        _loss = loss(z_, interval)
        acc_loss_original_corrupted[i] += _loss[indices].sum()
        mask = np.ones_like(_loss, dtype=bool)
        mask[indices] = False
        acc_loss_original_uncorrupted[i] = _loss[mask].sum()
print("Loss on corrupted: ", acc_loss_corrupted)
print("Loss on not corrupted: ", acc_loss_uncorrupted)
print("Loss on corrupted original: ", acc_loss_original_corrupted)
print("Loss on not corrupted original: ", acc_loss_original_uncorrupted)

What are some cuss words in english?
[0, 7]
loss: [0.5917583  0.40020847 0.04423489 0.15829569 0.29743534 0.1440694
 0.         0.7420018 ]
What kind of noises did dinosaurs make?
[6, 1]
loss: [0.         0.504419   0.01199764 0.06782848 0.0100733  0.
 0.6978543 ]
If you were going to steal from a convenience store, do you think it would be better in the morning or at night?
[7, 21]
loss: [0.         0.         0.         0.         0.         0.
 0.         0.2090655  0.28445116 0.         0.03544232 0.
 0.         0.         0.         0.         0.         0.
 0.         0.08997941 0.         0.74359196 0.18795347 0.        ]
Loss on corrupted:  [3.4886908531188965]
Loss on not corrupted:  [0.59782636]
Loss on corrupted original:  [0.0]
Loss on not corrupted original:  [0.0]
